## Create a personal story with my photo

## Basic Settings

In [10]:
!pip install httpx==0.27.2
!pip install openai
!pip install gradio
!pip install fpdf
!pip install python-docx

In [11]:
# import lines
import requests
import zipfile
import io
import os
import base64
import matplotlib.pyplot as plt
import time
import gradio as gr
import asyncio
import threading
from io import BytesIO
from openai import OpenAI
from PIL import Image
from IPython.display import display, HTML
from fpdf import FPDF  # PDF 생성 라이브러리
import docx  # Word 파일 생성 라이브러리
import webbrowser  # 웹 브라우저로 링크 열기 위해 사용

In [12]:
# 파일 ID와 다운로드 URL 리스트
sample_image_url1 = "https://drive.google.com/uc?id=1vuhPvNwlYqB4eRl0nvEPwcRoLnxy0mhx"
sample_image_url2 = "https://drive.google.com/uc?id=1tvV7pVSdi9HT7TT9rQgptqSr6g7cosJz"
sample_image_url3 = "https://drive.google.com/uc?id=1gWJS9PNEHT3R-eFmMhPlaOuBBZXbW0vA"
sample_image_url4 = "https://drive.google.com/uc?id=1ByigXqQ_JPRtcAeBlnSnl32iQUb-A3XQ"
sample_image_url5 = "https://drive.google.com/uc?id=1F2bpo8Z2sTqOyKPlcbBhuM-RRR69Lb6f"
sample_image_url6 = "https://drive.google.com/uc?id=1vyBIxeUbMPQ0FNe9GnC0jMlVZGsv3Tt_"
sample_image_url7 = "https://drive.google.com/uc?id=1A_-hUey9oYNj-eOga63sBO14KrsrkT2u"

image_lst = [
    sample_image_url1,
    sample_image_url2,
    sample_image_url3,
    sample_image_url4,
    sample_image_url5,
    sample_image_url6,
    sample_image_url7,
    ]

# 파일 ID와 다운로드 URL 리스트
file_info = {
    "17kzFOeDkZy-SUQmsIISTLQ7MHZjf0ThT": "sample1.zip", # 샘플파일1
    "1VEaM3Ltz5ZS87EbeFsgkDC6aM0s2iivy": "sample2", # 샘플파일2 건국대
    "1jeib4BD2aG38F0CPi053A1cTMjrgK4Aa": "page1_select_genre.zip", # 1p 장르선택 이미지파일
    "1K4FwYcpc-UBvkStpaUsW0j4clVAomzc6": "page2_select_setting.zip", # 2p 세팅선택 이미지파일
    "16oiuQogYqvquhRkqc44hQpIbVaXVh1uj": "page3_select_drawing_style.zip", # 3p 그림체선택 이미지파일
    "1tBeoFYyG-YQCFfLeuzKOUCrsPRsD7r7R": "home.zip" # 프로젝트 소개
}

base_url = "https://drive.google.com/uc?id="

# 압축 해제할 디렉토리
extract_root = "extracted_files"

# 디렉토리가 없으면 생성
if not os.path.exists(extract_root):
    os.makedirs(extract_root)

# 각 파일 다운로드 및 압축 해제
for file_id, file_name in file_info.items():
    url = f"{base_url}{file_id}"
    print(f"파일 다운로드 중... (ID: {file_id}, 이름: {file_name})")
    response = requests.get(url)

    if response.status_code == 200:
        # 파일 이름을 기준으로 디렉토리 생성
        folder_name = os.path.splitext(file_name)[0]  # 확장자 제거한 폴더 이름
        extract_path = os.path.join(extract_root, folder_name)

        if not os.path.exists(extract_path):
            os.makedirs(extract_path)

        print(f"파일 압축 해제 중... (파일 이름: {file_name})")
        try:
            with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
                zip_ref.extractall(extract_path)
            print(f"파일이 '{extract_path}' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: {file_name})")
        except zipfile.BadZipFile:
            print(f"파일이 압축 파일이 아닙니다. (파일 이름: {file_name})")
    else:
        print(f"파일 다운로드에 실패했습니다. (ID: {file_id}, 이름: {file_name})")

파일 다운로드 중... (ID: 17kzFOeDkZy-SUQmsIISTLQ7MHZjf0ThT, 이름: sample1.zip)
파일 압축 해제 중... (파일 이름: sample1.zip)
파일이 'extracted_files/sample1' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: sample1.zip)
파일 다운로드 중... (ID: 1VEaM3Ltz5ZS87EbeFsgkDC6aM0s2iivy, 이름: sample2)
파일 압축 해제 중... (파일 이름: sample2)
파일이 'extracted_files/sample2' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: sample2)
파일 다운로드 중... (ID: 1jeib4BD2aG38F0CPi053A1cTMjrgK4Aa, 이름: page1_select_genre.zip)
파일 압축 해제 중... (파일 이름: page1_select_genre.zip)
파일이 'extracted_files/page1_select_genre' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: page1_select_genre.zip)
파일 다운로드 중... (ID: 1K4FwYcpc-UBvkStpaUsW0j4clVAomzc6, 이름: page2_select_setting.zip)
파일 압축 해제 중... (파일 이름: page2_select_setting.zip)
파일이 'extracted_files/page2_select_setting' 디렉토리에 성공적으로 압축 해제되었습니다. (파일 이름: page2_select_setting.zip)
파일 다운로드 중... (ID: 16oiuQogYqvquhRkqc44hQpIbVaXVh1uj, 이름: page3_select_drawing_style.zip)
파일 압축 해제 중... (파일 이름: page3_select_drawing_style.zip)
파일이 'extracted_files/page3_select_drawing_style' 디렉토리

## API Settings

In [15]:
## API Settings
openai_api_key = os.getenv("OPENAI_API_KEY", "Please enter your API key here.")
client = OpenAI(api_key=openai_api_key)

## Run Program

In [14]:
# Step 1: Set genre selection image paths
gener_imagefile_dict = {
    "Fairy Tale": "/content/extracted_files/page1_select_genre/fairytale.webp",
    "Adventure": "/content/extracted_files/page1_select_genre/adventure.webp",
    "Fantasy": "/content/extracted_files/page1_select_genre/fantasy.webp",
    "Mystery": "/content/extracted_files/page1_select_genre/mystery.webp",
    "Sci-Fi": "/content/extracted_files/page1_select_genre/sf.webp",
    "Superhero": "/content/extracted_files/page1_select_genre/superhero.webp",
    "Animal Story": "/content/extracted_files/page1_select_genre/animal.webp",
    "Scary": "/content/extracted_files/page1_select_genre/scary.webp"
}

# Step 2: Set setting selection image paths
settings_imagefile_dict = {
    "Random": "/content/extracted_files/page2_select_setting/Random.webp",
    "Castle": "/content/extracted_files/page2_select_setting/Castle.webp",
    "School": "/content/extracted_files/page2_select_setting/School.webp",
    "Jungle": "/content/extracted_files/page2_select_setting/Jungle.webp",
    "Farm": "/content/extracted_files/page2_select_setting/Farm.webp",
    "City": "/content/extracted_files/page2_select_setting/City.webp",
    "Desert": "/content/extracted_files/page2_select_setting/Desert.webp",
    "Mountains": "/content/extracted_files/page2_select_setting/Mountains.webp",
    "Ocean": "/content/extracted_files/page2_select_setting/Ocean.webp",
    "Forest": "/content/extracted_files/page2_select_setting/Forest.webp",
    "Space": "/content/extracted_files/page2_select_setting/Space.webp",
    "Small Town": "/content/extracted_files/page2_select_setting/SmallTown.webp",
    "Island": "/content/extracted_files/page2_select_setting/Island.webp",
    "Snow": "/content/extracted_files/page2_select_setting/Snow.webp",
    "Underground": "/content/extracted_files/page2_select_setting/Underground.webp",
    "Village": "/content/extracted_files/page2_select_setting/Village.webp",
    "Amusement Park": "/content/extracted_files/page2_select_setting/AmusementPark.webp",
    "Zoo": "/content/extracted_files/page2_select_setting/Zoo.webp",
    "Futuristic City": "/content/extracted_files/page2_select_setting/FuturisticCity.webp",
    "Ancient City": "/content/extracted_files/page2_select_setting/AncientCity.webp"
}

# Step 3: Set drawing style selection image paths
drawing_style_imagefile_dict = {
    "Cartoon": "/content/extracted_files/page3_select_drawing_style/cartoon.webp",
    "Watercolor": "/content/extracted_files/page3_select_drawing_style/watercolor.webp",
    "Oil Painting": "/content/extracted_files/page3_select_drawing_style/oil_painting.webp",
    "Pixel Art": "/content/extracted_files/page3_select_drawing_style/pixel_art.webp",
    "Pencil Sketch": "/content/extracted_files/page3_select_drawing_style/pencil_sketch.webp",
    "Animation": "/content/extracted_files/page3_select_drawing_style/animation_style.webp",
    "Realistic": "/content/extracted_files/page3_select_drawing_style/realistic.webp",
    "Webtoon": "/content/extracted_files/page3_select_drawing_style/webtoon.webp"
}


#################### [Step 1: Genre Selection Related Functions] ####################
def select_genre(genre):
    # Used in Step 1: Genre Selection
    message = f"You have selected the '{genre}' genre. Proceed to the next step."
    return genre, message

#################### [Step 2: Setting Selection Related Functions] ####################
def select_setting(setting):
    # Used in Step 2: Setting Selection
    message = f"You have selected the '{setting}' setting. Proceed to the next step."
    return setting, message

#################### [Step 3: Drawing Style Selection Related Functions] ####################
def select_drawing_style(style):
    # Used in Step 3: Drawing Style Selection
    message = f"You have selected the '{style}' drawing style. Proceed to the next step."
    return style, message

#################### [Step 4: Character Selection Related Functions] ####################
def add_main_character(name, gender, age, traits):
    # Used in Step 4: Add Main Character
    character_info = {
        "name": name,
        "gender": gender,
        "age": age,
        "traits": traits
    }
    message = f"Main character added: Name: {name}, Gender: {gender}, Age: {age}, Traits: {traits}"
    return character_info, message

def random_character():
    # Used in Step 4: Generate Random Character
    name = "Kim Cheol-soo"
    gender = "Male"
    age = "25"
    traits = "Brave, Kind"
    return name, gender, age, traits

#################### [Step 5: Submit Story Ideas Related Functions] ####################
def submit_story_ideas(story_idea):
    # Used in Step 5: Submit Story Idea
    message = f"Story idea submitted: {story_idea}"
    return story_idea, message

#################### [Step 6: Story Length Selection Related Functions] ####################
def select_story_length(length):
    # Used in Step 6: Select Story Length
    message = f"Selected story length: {length}"
    return length, message

#################### [Step 7: Generate Story and Images Related Functions] ####################

def encode_image(image):
    # Used in Step 7: Image Encoding
    if image is None:
        raise ValueError("No image provided.")
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def format_character_info(character_info):
    # Used in Step 7: Format Character Information
    if character_info:
        return f"Name: {character_info.get('name', 'N/A')}, Gender: {character_info.get('gender', 'N/A')}, Age: {character_info.get('age', 'N/A')}, Traits: {character_info.get('traits', 'N/A')}"
    else:
        return "N/A"

def generate_image_description(image):
    # Used in Step 7: Generate Image Description
    max_retries = 5
    for attempt in range(max_retries):
        try:
            base64_image = encode_image(image)
            image_url = f"data:image/jpeg;base64,{base64_image}"
            description_prompt = """Please describe the provided image realistically in less than 200 characters based on the following elements:
background, position of the subjects (people or objects) in the photo, distinctive shapes, expressions, sizes, poses, surrounding environment, color tones, direction and intensity of light, mood."""
            start_time = time.time()
            response_text = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": f"{description_prompt}"
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"{image_url}",
                                },
                            },
                        ],
                    }
                ],
            )
            end_time = time.time()
            duration = end_time - start_time
            print(f"Image description generation time: {duration:.2f} seconds")
            output_text = response_text.choices[0].message.content
            return output_text

        except Exception as e:
            print(f"Error occurred (Attempt {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                print("Reached maximum retry attempts. Failed to generate image description.")
                return None
            else:
                print("Retrying...\n")

def generate_text_from_image(image, genre, setting, character_info, story_idea, story_length, chapter_number, image_description, previous_story=None):
    # Used in Step 7: Generate Story Based on Uploaded Image

    if image_description is None:
        print("Failed to generate image description. Cannot generate story.")
        return None, None

    base64_image = encode_image(image)
    genre_text = genre if genre else "Random"
    setting_text = setting if setting else "Random"
    character_text = format_character_info(character_info)
    story_idea_text = story_idea if story_idea else "N/A"
    story_length_text = story_length if story_length else "Standard length, approximately 750-1050 words"
    previous_story_text = previous_story if previous_story else "N/A"

    max_retries = 5
    for attempt in range(max_retries):
        try:
            system_prompt = """
You are a creative story writer. Based on the provided image and its description, along with the instructions and guidelines, please write an engaging and captivating story.
            """
            user_prompt = f"""
[Image Description]
{image_description}

[Instructions]
- Please refer to the information below to write the story.
- Seamlessly incorporate the details of the uploaded image into the story and describe them in detail.
- The story should have the mood and tone that matches the selected genre and setting.
- The content to be written is the next chapter (Chapter {chapter_number}) of the story. One chapter will be created for each image.
- Elements that must be included in the story:
  - Genre: {genre_text}
  - Setting: {setting_text}
  - Main Character Information: {character_text}
  - Story Idea: {story_idea_text}
- Please write the story to match the length of {story_length_text}.

[Previous Story]
{previous_story_text}

[Guidelines]
- If there is a previous story, ensure that the new chapter flows naturally from the previous content.
- Maintain the same traits and backgrounds for characters that appeared in the previous story.
- At the beginning of each chapter, indicate "Chapter {chapter_number}".
- Replace brand names or specific institution names with general expressions.
- Avoid inappropriate or offensive content.
- Use literary expressions and rich descriptions to stimulate the reader's imagination.
"""
            print(f"Story generation prompt: {user_prompt}")
            messages = [
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}",
                            },
                        },
                        {
                            "type": "text",
                            "text": user_prompt
                        },
                    ],
                },
            ]

            start_time = time.time()
            response_text = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
            )
            end_time = time.time()
            duration = end_time - start_time
            print(f"Story generation time: {duration:.2f} seconds")

            output_text = response_text.choices[0].message.content
            return output_text

        except Exception as e:
            print(f"Error occurred (Attempt {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                print("Reached maximum retry attempts. Failed to generate story.")
                return None
            else:
                print("Retrying...\n")

def generate_image_only_from_story(previous_chapters_story, genre, setting, character_info, story_idea, story_length, image_description, drawing_style):
    # Used in Step 7: Generate New Image Based on Generated Story
    genre_text = genre if genre else "Random"
    setting_text = setting if setting else "Random"
    character_text = format_character_info(character_info)
    drawing_style_text = drawing_style if drawing_style else "Freely"

    max_retries = 5
    for attempt in range(max_retries):
        try:
            user_prompt = f"""
[Image Description]
{image_description}
[Instructions]
- You are an excellent illustrator.
- Create a high-quality illustration based on the **Image Description** above.
- The drawing should consider the following elements:
  - Genre: {genre_text}
  - Setting: {setting_text}
  - Drawing Style: {drawing_style_text}
  - Main Character Settings: {character_text}
- The mood and style of the drawing must reflect the specified genre, setting, drawing style, and main character settings.
[Guidelines]
- Visually represent the main character's traits and emotions well.
- Effectively convey the atmosphere using colors and lighting.
- Do not include any text or letters in the image.
"""
            print(f"Image generation prompt: {user_prompt}")
            start_time = time.time()
            response_generated_image = client.images.generate(
                model="dall-e-3",
                prompt=user_prompt,
                size="1792x1024",
                quality="hd",
                n=1,
            )
            end_time = time.time()
            duration = end_time - start_time
            print(f"Image generation time: {duration:.2f} seconds")

            response = requests.get(response_generated_image.data[0].url)
            if response.status_code == 200:
                return Image.open(BytesIO(response.content))
            else:
                print(f"Failed to download image. Status code: {response.status_code}")
                return None

        except Exception as e:
            print(f"Error occurred (Attempt {attempt + 1}/{max_retries}): {e}")
            if attempt == max_retries - 1:
                print("Reached maximum retry attempts. Failed to generate image.")
                return None
            else:
                print("Retrying...\n")

async def handle_image_upload(image, story_state, image_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, chapters_state, drawing_style_state):
    # Used in Step 7: Generate Story + Image upon Image Upload
    if image is None:
        return "Please upload an image.", None, story_state, image_state, [], None, None, chapters_state

    chapter_number = len(chapters_state) + 1
    previous_chapter = chapters_state[-1] if chapters_state else None

    start_time = time.time()
    image_description = generate_image_description(image)

    # Run two functions simultaneously (convert synchronous functions to asynchronous)
    output_text, generated_image = await asyncio.gather(
        asyncio.to_thread(
            generate_text_from_image,
            image,
            genre_state,
            setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            chapter_number,
            image_description,
            previous_chapter
        ),
        asyncio.to_thread(
            generate_image_only_from_story,
            previous_chapter,
            genre_state, setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            image_description,
            drawing_style_state
        )
    )
    end_time = time.time()
    duration = end_time - start_time
    print(f"Total story + image generation time: {duration:.2f} seconds")

    if output_text is None or generated_image is None:
        return "Failed to generate story.", None, story_state, image_state, [], None, None, chapters_state

    image_description_state = image_description
    new_chapters_state = chapters_state + [output_text]
    new_story_state = '\n'.join(new_chapters_state)
    new_previous_story_state = output_text
    new_image_state = image_state + [generated_image] if generated_image else image_state
    gallery_images = [img for img in new_image_state if img is not None]

    return new_story_state.replace('\\n', '\n'), generated_image, new_story_state, new_image_state, gallery_images, image_description_state, new_previous_story_state, new_chapters_state

async def regenerate_story(image_state, story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_description_state, chapters_state, drawing_style_state):
    # Used in Step 7: Regenerate Story
    if not image_state:
        return "No image available.", None, story_state, image_state, [], image_description_state, previous_story_state, chapters_state

    last_image = image_state[-1]
    chapter_number = len(chapters_state)
    previous_chapter = chapters_state[-2] if len(chapters_state) > 1 else None
    image_description = image_description_state

    # Run two functions simultaneously
    output_text, generated_image = await asyncio.gather(
        asyncio.to_thread(
            generate_text_from_image,
            last_image,
            genre_state,
            setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            chapter_number,
            image_description,
            previous_story=previous_chapter
        ),
        asyncio.to_thread(
            generate_image_only_from_story,
            previous_chapter,
            genre_state,
            setting_state,
            character_state,
            story_idea_state,
            story_length_state,
            image_description,
            drawing_style_state
        )
    )

    if generated_image is None or output_text is None:
        return "Failed to regenerate story.", None, story_state, image_state, [], image_description_state, previous_story_state, chapters_state

    new_chapters_state = chapters_state[:-1] + [output_text]
    new_story_state = '\n'.join(new_chapters_state)
    new_image_description_state = image_description
    new_image_state = image_state.copy()
    if generated_image:
        new_image_state[-1] = generated_image

    gallery_images = [img for img in new_image_state if img is not None]

    return new_story_state.replace('\\n', '\n'), generated_image, new_story_state, new_image_state, gallery_images, new_image_description_state, output_text, new_chapters_state

def regenerate_image(story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_state, image_description_state, drawing_style_state):
    # Used in Step 7: Regenerate Image
    generated_image = generate_image_only_from_story(previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_description_state, drawing_style_state)

    new_image_state = image_state.copy()
    if generated_image is not None:
        new_image_state[-1] = generated_image

    gallery_images = [img for img in new_image_state if img is not None]

    return generated_image, new_image_state, gallery_images

#################### [Final: Export and Share Related Functions] ####################

def export_to_pdf(story_text, image_list):
    # Export the completed story to PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_font('NotoSans', '', 'NotoSansCJKkr-Regular.otf', uni=True)
    pdf.set_font('NotoSans', size=12)

    story_paragraphs = story_text.split('\n')
    for idx in range(max(len(image_list), len(story_paragraphs))):
        pdf.add_page()
        if idx < len(image_list) and image_list[idx]:
            img_path = f"temp_image_{idx}.png"
            image_list[idx].save(img_path)
            pdf.image(img_path, x=10, y=10, w=pdf.w - 20)
            os.remove(img_path)
            pdf.ln(85)
        if idx < len(story_paragraphs):
            pdf.multi_cell(0, 10, story_paragraphs[idx], align='L')

    pdf_output = io.BytesIO()
    pdf.output(pdf_output)
    pdf_output.seek(0)
    return pdf_output

def export_to_word(story_text, image_list):
    # Export the completed story to Word
    doc = docx.Document()
    story_paragraphs = story_text.split('\n')

    for idx in range(max(len(image_list), len(story_paragraphs))):
        if idx < len(image_list) and image_list[idx]:
            img_path = f"temp_image_{idx}.png"
            image_list[idx].save(img_path)
            doc.add_picture(img_path, width=docx.shared.Inches(6))
            os.remove(img_path)
        if idx < len(story_paragraphs):
            doc.add_paragraph(story_paragraphs[idx])

    word_output = io.BytesIO()
    doc.save(word_output)
    word_output.seek(0)
    return word_output

def share_on_kakaotalk(story_text):
    # Share the completed story on KakaoTalk
    share_text = f"My Story:\n{story_text}"
    kakao_link = f"https://sharer.kakao.com/talk/friends/picker/link?link_ver=4&text={requests.utils.quote(share_text)}"
    webbrowser.open(kakao_link)

def share_on_twitter(story_text):
    # Share the completed story on Twitter
    share_text = f"My Story:\n{story_text}"
    twitter_link = f"https://twitter.com/intent/tweet?text={requests.utils.quote(share_text)}"
    webbrowser.open(twitter_link)

#################### UI and Execution Section ####################
with gr.Blocks(css=".gradio-container footer {display: none !important;}") as demo:

    # State variables
    genre_state = gr.State()
    setting_state = gr.State()
    character_state = gr.State()
    story_idea_state = gr.State()
    story_length_state = gr.State()
    previous_story_state = gr.State("")
    image_description_state = gr.State()
    image_state = gr.State([])  # Initialize as a list of images
    story_state = gr.State("")  # Initialize as an empty story string
    chapters_state = gr.State([])  # List to store each chapter's content
    drawing_style_state = gr.State()  # Add drawing style state variable

    #################### Start Tab Configuration ####################

    with gr.Tabs(elem_id="tabs") as tabs:
        # Project Introduction Tab
        with gr.TabItem("Project Introduction"):

            # Start Button (Top)
            start_button_top = gr.Button("Get Started", variant="primary")
            start_button_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )

            # Project Introduction Section
            gr.Image(value="https://drive.google.com/uc?id=10WVpFeZ3_8KxfBWg5OZG-yKUQY0z9Au2")
            gr.Markdown("# Project Introduction")
            gr.Markdown("""
            [View Project Report](https://drive.google.com/uc?id=1MN97nvxdc4gzdN486hdPiO25bEw07cPj)
            [Download Sample Files](https://drive.google.com/uc?id=1VEaM3Ltz5ZS87EbeFsgkDC6aM0s2iivy)
            """)
            gr.Markdown("""
            ## Create Your Own Story with Your Photos
            ### AI-Based Interactive Story Creation Platform
                This project combines your photos and imagination with AI to create personalized stories and visually reproduce them as images, providing a new digital content experience.
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=1ufnovxeKPbNoulGPDIgmjrsWtIRc_YCq")
            gr.Markdown("""
            ## Project Overview
            ### Project Background
            In the digital age, people record and share moments through various media such as photos, videos, and text. However, the desire to expand memories and emotions beyond mere appreciation is growing.
            This project reflects these needs by utilizing large language models (LLMs) like **ChatGPT** and image generation AIs to propose an interactive platform where users can create unique stories using their **own photos + imagination**.
            """)

            gr.Markdown("""
            ### Goals and Expected Effects
            - **Personalized Storytelling Experience**: User-driven settings including photos, genre, background, drawing style, characters, and length.
            - **Creativity Enhancement**: Expanding stories based on photos to boost creative thinking and immersion.
            - **Improved Technology Accessibility**: Implementing AI through an intuitive UI/UX, making it easy for anyone to use.
            - **New Digital Content Ecosystem**: Potential expansion into various fields such as education, arts, and marketing.
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=1m0FzoDf5lpKhjQI_0YA0hdoau-IZYnTZ")
            gr.Image(value="https://drive.google.com/uc?id=1EoiTjcU_4xTnckslokMv_qUxwDtJMe17")
            gr.Markdown("""
            ## AI-Based Story and Image Generation Process
            The above diagram shows the step-by-step AI utilization flow within the project:
                Image Upload → Image Description Generation → Story Prompt Construction → Story Generation → Illustration Generation
            """)

            gr.Markdown("""
            ## Key Features
            1. **Photo Upload and Analysis**
            - Analyze user photos with AI models to generate textual descriptions.
            - Utilize these descriptions for story generation prompts.

            2. **Genre, Background, Drawing Style, Character Settings**
            - Genre: Variety of genres like Fairy Tale, Fantasy, Sci-Fi, Mystery, etc.
            - Background: Specify desired environments such as castle, forest, city, space, etc.
            - Drawing Style: Choose styles like Cartoon, Watercolor, Pixel Art, etc.
            - Characters: Freely set names, traits, age, gender, etc.

            3. **Input Story Ideas and Adjust Length**
            - Enter user-provided story ideas or specific settings.
            - Select from short to long narratives.

            4. **AI-Based Story and Image Generation**
            - Automatically generate customized stories using ChatGPT (LLM).
            - Automatically generate story illustrations using models like Stable Diffusion and DALL·E.
            - Regeneration feature allows users to achieve desired results.

            5. **Export and Share Results**
            - Download completed stories and images as PDF or Word files.
            - Support sharing on SNS platforms like KakaoTalk and Twitter.
            """)

            gr.Image(value="https://drive.google.com/uc?id=1RO47hCP2SybG-BHPOTxabcd9_gSyzXq4")
            gr.Markdown("""
            ## Interface and User Flow
                Users proceed through the following steps: Select Genre → Select Setting → Select Drawing Style → Input Story Idea → Select Story Length → Upload Image → Generate Story and Image.
            ---
            """)

            gr.Markdown("""
            ## Implementation Plan and Technology Stack
            - **Frontend/UI**: Gradio-based web interface
            Intuitive and supports step-by-step decision-making flow.
            - **Backend Logic**: Python-based server
            Handles image processing, API integration, and state management.
            - **AI Model Integration**: OpenAI API (ChatGPT), Stable Diffusion, CLIP/BLIP, etc.
            Utilizes Vision-Language models.
            - **Scalability**: Modular API structure allows integration with various models and services.
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=12N9g_X_qP4fVywvLVNKdTkxfWhIx-K7O")
            gr.Markdown("""
            ## Initial Strategy and Growth Plan
            - **Micro-Market Strategy**: Target niche markets such as educational customized fairy tales or specific genre enthusiasts.
            - **Surface-Level First Strategy**: Release an MVP centered around core features (photo-based story) and gradually expand.
            - **Producer-Led Spread Strategy**: Encourage users to share their creations on SNS, fostering organic word-of-mouth.
            ---
            """)

            gr.Image(value="https://drive.google.com/uc?id=1QRT2rbR6fPHvwyzXyNkKfrtv0r_MBuzt")
            gr.Markdown("""
            ## Expected Effects
            - **Immersive Digital Storytelling Experience**: Provides new content by combining photos and AI.
            - **Maximized User Creativity**: Rich creative process through immediate feedback and regeneration.
            - **Sustainable Ecosystem Creation**: Applicable and expandable in various fields such as education, marketing, and arts.
            ---
            """)

            gr.Markdown("""
            ## Conclusion and Future Tasks

            This project proposes a new paradigm in personalized interactive storytelling. Future plans include enhancing models, improving UI/UX, collaborating with educational and cultural art sectors, and exploring monetization models to further advance the platform.
            """)

            # Start Button (Bottom)
            start_button_bottom = gr.Button("Get Started", variant="primary")
            start_button_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )

        # Step 1: Genre Selection
        with gr.TabItem("Select Genre"):
            gr.Markdown("# Create a New Story")
            gr.Markdown("## Start Your Story: Choose a Genre")
            gr.Markdown("Every great story starts with a genre. Choose a genre that sets the mood and style for your adventure.")

            # Top Back and Next Buttons (Back disabled for Step 1)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn1_top = gr.Button("Back", variant="secondary", interactive=False)
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn1_top = gr.Button("Next", variant="primary")

            next_btn1_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )

            # Textbox to display selected genre
            selected_genre_text = gr.Textbox(label="Selected Genre", interactive=False)

            # List of genres and descriptions
            genres = ["Fairy Tale", "Adventure", "Fantasy", "Mystery", "Sci-Fi", "Superhero", "Animal Story", "Scary"]
            genre_descriptions = [
                "Traditional stories often featuring magic, princes and princesses, and moral lessons.",
                "Exciting journeys, quests, or explorations in exotic or challenging environments.",
                "Stories involving magical elements, mythical creatures, and fantastical worlds.",
                "Stories where characters solve puzzles, unravel mysteries, or uncover secrets.",
                "Focuses on futuristic technology, space exploration, time travel, and other scientific concepts.",
                "Stories featuring characters with extraordinary abilities or powers fighting against evil.",
                "Main characters are animals with human-like traits, exploring their adventures or interactions.",
                "Scary stories that may include ghosts, monsters, or zombies, providing a bit of suspense."
            ]

            with gr.Row():
                for i, genre in enumerate(genres):
                    with gr.Column():
                        image_path = gener_imagefile_dict.get(genre)
                        if image_path and os.path.exists(image_path):
                            img = Image.open(image_path)
                            gr.Image(img, label=genre, elem_id=f"{genre.lower().replace(' ', '_')}_image")
                        else:
                            gr.Markdown(f"**Image not available**")

                        gr.Markdown(f"**{genre}**\n\n{genre_descriptions[i]}")
                        btn = gr.Button("Select")
                        btn.click(fn=lambda g=genre: select_genre(g), inputs=None, outputs=[genre_state, selected_genre_text], queue=False)

            # Bottom Back and Next Buttons (Back disabled for Step 1)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn1_bottom = gr.Button("Back", variant="secondary", interactive=False)
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn1_bottom = gr.Button("Next", variant="primary")

            next_btn1_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )

        # Step 2: Setting Selection
        with gr.TabItem("Select Setting"):
            gr.Markdown("## Scene Setting: Choose a Setting")
            gr.Markdown("Where does your story take place? From the bustling streets of a city to the serene landscapes of a distant planet, choose a setting that brings your story to life.")

            selected_setting_text = gr.Textbox(label="Selected Setting", interactive=False)

            # Top Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn2_top = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn2_top = gr.Button("Next", variant="primary")

            back_btn2_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 0) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )
            next_btn2_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )

            settings = [
                "Random", "Castle", "School", "Jungle", "Farm", "City", "Desert", "Mountains",
                "Ocean", "Forest", "Space", "Small Town", "Island", "Snow",
                "Underground", "Village", "Amusement Park", "Zoo", "Futuristic City", "Ancient City"
            ]
            setting_descriptions = [
                "Randomly selects one of the predefined story settings (castle, island, amusement park, etc.).",
                "A historical or fantastical setting with kingdoms or medieval themes, often used in fairy tales or fantasy stories.",
                "An educational space suitable for stories about friendship, learning, and personal growth.",
                "A dense tropical environment filled with exotic flora and fauna, ideal for adventure and exploration stories.",
                "A rural environment emphasizing nature and agricultural life, with fields, barns, and animals.",
                "An urban environment with skyscrapers and diverse communities, reflecting a fast-paced modern lifestyle.",
                "A harsh, dry environment filled with sand, suitable as a backdrop for survival and adventure stories.",
                "A mountainous setting with rugged terrain, frequently used in adventure and exploration themes.",
                "A marine environment including beaches, secluded or underwater settings, perfect for exploration and adventure.",
                "A lush and possibly mystical forest area, ideal for fairy tales or adventure stories.",
                "An extraterrestrial setting on a spaceship or another planet, suitable for science fiction or futuristic stories.",
                "A close-knit community setting with a quaint atmosphere, ideal for stories about community and relationships.",
                "A remote, isolated environment perfect for adventure, survival, or mystery stories.",
                "A cold, snowy environment like the polar regions or snowy mountain ranges, suitable for survival and adventure story backgrounds.",
                "An underground setting such as caves or tunnels, providing a mysterious and adventurous atmosphere.",
                "A small, quiet community with local life, ideal for stories centered around community dynamics.",
                "A fun and exciting environment with rides and attractions, perfect for adventure and humorous stories.",
                "An environment with various animals, suitable for educational, animal-based, or adventure stories.",
                "A high-tech city environment with advanced technological features, suitable for science fiction or dystopian stories.",
                "Explore the mysteries and wonders of an ancient city by traveling back in time."
            ]

            with gr.Row():
                for i, setting in enumerate(settings):
                    with gr.Column():
                        image_path = settings_imagefile_dict.get(setting)
                        if image_path and os.path.exists(image_path):
                            img = Image.open(image_path)
                            gr.Image(img, label=setting)
                        else:
                            gr.Markdown("**Image not available**")

                        gr.Markdown(f"**{setting}**\n\n{setting_descriptions[i]}")
                        btn = gr.Button("Select")
                        btn.click(fn=lambda s=setting: select_setting(s), inputs=None, outputs=[setting_state, selected_setting_text], queue=False)

            # Bottom Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn2_bottom = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn2_bottom = gr.Button("Next", variant="primary")

            back_btn2_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 0) {
                            buttons[1].click();
                        }
                    }
                }
                """
            )

            next_btn2_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )


        # Step 3: Drawing Style Selection
        with gr.TabItem("Select Drawing Style"):
            gr.Markdown("## Select Drawing Style")
            gr.Markdown("Choose the drawing style to be used in your story.")
            selected_style_text = gr.Textbox(label="Selected Drawing Style", interactive=False)

            # Top Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn_style_top = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn_style_top = gr.Button("Next", variant="primary")

            back_btn_style_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )
            next_btn_style_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )

            drawing_styles = ["Cartoon", "Watercolor", "Oil Painting", "Pixel Art", "Pencil Sketch", "Animation", "Realistic", "Webtoon"]
            style_descriptions = [
                "Bright and vivid colors with simplified character depictions.",
                "Soft and blurred edges typical of watercolor paintings.",
                "Thick textures and rich colors characteristic of oil paintings.",
                "Classic game-style pixel art.",
                "Detailed pencil line sketches.",
                "Dynamic and expressive animation style.",
                "Realistic textures typical of real-life illustrations.",
                "Modern and digital webtoon style."
            ]

            with gr.Row():
                for i, style in enumerate(drawing_styles):
                    with gr.Column():
                        image_path = drawing_style_imagefile_dict.get(style)
                        if image_path and os.path.exists(image_path):
                            img = Image.open(image_path)
                            gr.Image(img, label=style)
                        else:
                            gr.Markdown("**Image not available**")

                        gr.Markdown(f"**{style}**\n\n{style_descriptions[i]}")
                        btn = gr.Button("Select")
                        btn.click(fn=lambda s=style: select_drawing_style(s), inputs=None, outputs=[drawing_style_state, selected_style_text], queue=False)

            # Bottom Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn_style_bottom = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn_style_bottom = gr.Button("Next", variant="primary")

            back_btn_style_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 1) {
                            buttons[2].click();
                        }
                    }
                }
                """
            )
            next_btn_style_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )

        # Step 4: Character Selection
        with gr.TabItem("Select Character"):
            gr.Markdown("## Add Character: Create Your Main Character")
            gr.Markdown("Introduce the characters in your story. Add your main character and surrounding characters, and define their traits to create rich and dynamic interactions.")

            # Top Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn3_top = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn3_top = gr.Button("Next", variant="primary")

            back_btn3_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )
            next_btn3_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )

            name_input = gr.Textbox(label="Name", placeholder="Enter the name")
            gender_input = gr.Dropdown(["Male", "Female", "Other"], label="Gender")
            age_input = gr.Textbox(label="Age", placeholder="Enter the age")
            traits_input = gr.Textbox(label="Traits", placeholder="Enter traits (e.g., Brave, Kind)")

            random_button = gr.Button("Generate Random")
            random_button.click(
                fn=random_character,
                inputs=None,
                outputs=[name_input, gender_input, age_input, traits_input]
            )

            character_info_display = gr.Textbox(label="Main Character Information", interactive=False)
            add_character_button = gr.Button("Add Main Character")
            add_character_button.click(
                fn=add_main_character,
                inputs=[name_input, gender_input, age_input, traits_input],
                outputs=[character_state, character_info_display]
            )

            # Bottom Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn3_bottom = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn3_bottom = gr.Button("Next", variant="primary")

            back_btn3_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 2) {
                            buttons[3].click();
                        }
                    }
                }
                """
            )
            next_btn3_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )

        # Step 5: Add Details
        with gr.TabItem("Add Details"):
            gr.Markdown("## Create Your Own Story: Story Ideas")
            gr.Markdown("### Story Ideas (Optional)")
            gr.Markdown("What special events would you like to happen in your story? You can add interesting moments or surprising twists here to shape your story the way you want!")

            # Top Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn4_top = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn4_top = gr.Button("Next", variant="primary")

            back_btn4_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )
            next_btn4_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )

            story_idea_input = gr.Textbox(
                placeholder="Enter interesting or exciting content for your story.",
                lines=4,
                show_label=False
            )
            story_output = gr.Textbox(label="Submitted Story Idea", interactive=False)
            continue_button = gr.Button("Continue")
            continue_button.click(
                fn=submit_story_ideas,
                inputs=story_idea_input,
                outputs=[story_idea_state, story_output]
            )

            # Bottom Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn4_bottom = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn4_bottom = gr.Button("Next", variant="primary")

            back_btn4_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 3) {
                            buttons[4].click();
                        }
                    }
                }
                """
            )
            next_btn4_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )

        # Step 6: Story Length Selection
        with gr.TabItem("Select Story Length"):
            gr.Markdown("## Select Story Length")
            gr.Markdown("Choose the desired length of your story.")
            selected_length_text = gr.Textbox(label="Selected Story Length", interactive=False)

            # Top Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn5_top = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn5_top = gr.Button("Next", variant="primary")

            back_btn5_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )
            next_btn5_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### Short Story")
                    gr.Markdown("Approximately 300-450 words")
                    short_button = gr.Button("Select")
                    short_button.click(
                        fn=lambda: select_story_length("Short length, approximately 300-450 words"),
                        inputs=[],
                        outputs=[story_length_state, selected_length_text]
                    )

                with gr.Column():
                    gr.Markdown("### Normal Story")
                    gr.Markdown("Approximately 750-1050 words")
                    normal_button = gr.Button("Select")
                    normal_button.click(
                        fn=lambda: select_story_length("Standard length, approximately 750-1050 words"),
                        inputs=[],
                        outputs=[story_length_state, selected_length_text]
                    )

                with gr.Column():
                    gr.Markdown("### Long Story 🔒")
                    gr.Markdown("Approximately 1500-2250 words \nAvailable only to logged-in users.")
                    long_button = gr.Button("Select", interactive=False)

            # Bottom Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn5_bottom = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn5_bottom = gr.Button("Next", variant="primary")

            back_btn5_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 4) {
                            buttons[5].click();
                        }
                    }
                }
                """
            )
            next_btn5_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

        # Step 7: Generate Images and Story
        with gr.TabItem("Generate Images and Story"):
            gr.Markdown("## Generating Images and Stories")
            gr.Markdown("Upload an image to generate a story based on it, and generate new images from that story.")

            # Top Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn6_top = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn6_top = gr.Button("Next", variant="primary")

            back_btn6_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )
            next_btn6_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 7) {
                            buttons[8].click();
                        }
                    }
                }
                """
            )

            image_input = gr.Image(type="pil", label="Upload Image")
            image_output = gr.Image(type="pil", label="Generated Image")
            text_output = gr.Textbox(label="Generated Story", lines=10, interactive=False)
            gallery_output = gr.Gallery(label="Previously Generated Images", columns=4, height='auto')

            image_input.change(
                fn=handle_image_upload,
                inputs=[image_input, story_state, image_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, chapters_state, drawing_style_state],
                outputs=[text_output, image_output, story_state, image_state, gallery_output, image_description_state, previous_story_state, chapters_state],
                show_progress=True
            )

            regenerate_story_btn = gr.Button("Regenerate Story")
            regenerate_image_btn = gr.Button("Regenerate Image")

            regenerate_story_btn.click(
                fn=regenerate_story,
                inputs=[image_state, story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_description_state, chapters_state, drawing_style_state],
                outputs=[text_output, image_output, story_state, image_state, gallery_output, image_description_state, previous_story_state, chapters_state],
            )

            regenerate_image_btn.click(
                fn=regenerate_image,
                inputs=[story_state, previous_story_state, genre_state, setting_state, character_state, story_idea_state, story_length_state, image_state, image_description_state, drawing_style_state],
                outputs=[image_output, image_state, gallery_output],
            )

            # Bottom Back and Next Buttons
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn6_bottom = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")
                with gr.Column(scale=1):
                    next_btn6_bottom = gr.Button("Next", variant="primary")

            back_btn6_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 5) {
                            buttons[6].click();
                        }
                    }
                }
                """
            )
            next_btn6_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 7) {
                            buttons[8].click();
                        }
                    }
                }
                """
            )

        # Step 8: Final Story Confirmation and Export
        with gr.TabItem("Confirm and Export Story"):
            gr.Markdown("## Final Story")
            gr.Markdown("Review the generated images and story, then save or share them.")

            # Top Back Button (No Next Button for the Final Step)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn7_top = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")

            back_btn7_top.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

            final_story_output = gr.Textbox(label="Final Story", lines=10, interactive=False)
            final_gallery_output = gr.Gallery(label="Generated Images", columns=4, height='auto')

            def display_final_story(story_state, image_state):
                return story_state, image_state

            display_button = gr.Button("View Final Story")
            display_button.click(
                fn=display_final_story,
                inputs=[story_state, image_state],
                outputs=[final_story_output, final_gallery_output]
            )

            pdf_export_button = gr.Button("Export as PDF")
            pdf_file = gr.File(label="Download PDF File")
            pdf_export_button.click(
                fn=export_to_pdf,
                inputs=[story_state, image_state],
                outputs=pdf_file
            )

            word_export_button = gr.Button("Export as Word")
            word_file = gr.File(label="Download Word File")
            word_export_button.click(
                fn=export_to_word,
                inputs=[story_state, image_state],
                outputs=word_file
            )

            kakaotalk_share_button = gr.Button("Share on KakaoTalk")
            kakaotalk_share_button.click(
                fn=share_on_kakaotalk,
                inputs=[story_state],
                outputs=None
            )

            twitter_share_button = gr.Button("Share on Twitter")
            twitter_share_button.click(
                fn=share_on_twitter,
                inputs=[story_state],
                outputs=None
            )

            # Bottom Back Button (No Next Button for the Final Step)
            with gr.Row():
                with gr.Column(scale=1):
                    back_btn7_bottom = gr.Button("Back", variant="secondary")
                with gr.Column(scale=6):
                    gr.Markdown("")

            back_btn7_bottom.click(
                None,
                [],
                [],
                js="""
                () => {
                    const tabs = document.getElementById('tabs');
                    if (tabs) {
                        const buttons = tabs.querySelectorAll('[role="tab"]');
                        if (buttons && buttons.length > 6) {
                            buttons[7].click();
                        }
                    }
                }
                """
            )

    #################### End Tab Configuration ####################
    demo.launch(debug=True, show_api=False)

# Project Introduction Section

![Project Introduction](https://drive.google.com/uc?id=10WVpFeZ3_8KxfBWg5OZG-yKUQY0z9Au2)

## Project Introduction

### Create Your Own Story with Your Photos  
#### AI-Based Interactive Storytelling Platform

This project combines users' photos and imagination with AI to create personalized stories and visually render them, offering a new digital content experience.

---

![Project Overview](https://drive.google.com/uc?id=1ufnovxeKPbNoulGPDIgmjrsWtIRc_YCq)

## Project Overview

### Project Background

In the digital age, people record and share moments through diverse media such as photos, videos, and text. However, there is a growing desire to expand the memories and emotions within those images beyond mere appreciation. This project reflects these needs by leveraging large language models (LLMs) like **ChatGPT** and image generation AI to propose an interactive platform where users can create unique stories using **their own photos + imagination**.

### Goals and Expected Outcomes

- **Personalized Storytelling Experience**: User-driven settings for photos, genres, backgrounds, art styles, characters, and story length.
- **Creativity Enhancement**: Stimulates creative thinking and engagement by reinterpreting photo-based stories.
- **Improved Accessibility to Technology**: Implements AI with intuitive UI/UX, making it accessible to everyone.
- **New Digital Content Ecosystem**: Potential applications in education, culture, arts, marketing, and more.

---

![AI Utilization Process 1](https://drive.google.com/uc?id=1m0FzoDf5lpKhjQI_0YA0hdoau-IZYnTZ)  
![AI Utilization Process 2](https://drive.google.com/uc?id=1EoiTjcU_4xTnckslokMv_qUxwDtJMe17)

## AI-Based Story and Image Generation Process

The diagrams illustrate step-by-step AI utilization in the project:  
Image Upload → Image Description Generation → Story Prompt Configuration → Story Generation → Illustration Generation

### Key Features

1. **Photo Upload and Analysis**  
   - Analyzes user photos with AI to generate text descriptions.  
   - Utilized in story generation prompts.

2. **Genre, Background, Art Style, and Character Settings**  
   - Genres: Fairy tale, fantasy, sci-fi, mystery, and more.  
   - Backgrounds: Castle, forest, city, space, and other desired environments.  
   - Art styles: Cartoon, watercolor, pixel art, and more.  
   - Characters: Customize name, traits, age, gender, and more.

3. **Story Idea Input and Length Adjustment**  
   - Accepts user input for ideas or specific settings.  
   - Options for short stories to long narratives.

4. **AI-Based Story and Image Generation**  
   - Custom story generation powered by ChatGPT (LLM).  
   - Automatic story illustration generation using Stable Diffusion, DALL·E, etc.  
   - Regeneration feature for achieving desired outcomes.

5. **Sharing and Exporting Results**  
   - Download completed story and images in PDF or Word format.  
   - Supports sharing on social platforms like KakaoTalk, Twitter, and more.

---

![Interface Workflow](https://drive.google.com/uc?id=1RO47hCP2SybG-BHPOTxabcd9_gSyzXq4)

## Interface and User Workflow

Users proceed as follows:  
Select Genre → Choose Settings → Select Art Style → Enter Story Idea → Choose Story Length → Upload Image → Generate Story and Image

---

## Implementation Plan and Technology Stack

- **Frontend/UI**: Gradio-based web interface  
  Supports intuitive, step-by-step decision-making flow.  
- **Backend Logic**: Python-based server  
  Handles image processing, API integration, and state management.  
- **AI Model Integration**: OpenAI API (ChatGPT), Stable Diffusion, CLIP/BLIP, etc.  
  Utilizes vision-language models.  
- **Scalability**: Modular API structure allows integration with various models and services.

---

![Initial Strategy and Growth Plan](https://drive.google.com/uc?id=12N9g_X_qP4fVywvLVNKdTkxfWhIx-K7O)

## Initial Strategy and Growth Plan

- **Micro-Market Strategy**: Target niche markets such as custom educational fairy tales and genre enthusiasts.
- **Core Feature Focus Strategy**: Launch an MVP centered on the key feature (photo-based storytelling) and gradually expand.
- **User-Driven Promotion Strategy**: Encourage users to share their creations on social media, fostering organic word-of-mouth.

---

![Expected Outcomes](https://drive.google.com/uc?id=1QRT2rbR6fPHvwyzXyNkKfrtv0r_MBuzt)

## Expected Outcomes

- **Immersive Digital Storytelling Experience**: Provides innovative content by combining photos and AI.
- **Maximized User Creativity**: Enriches the creative process with instant feedback and regeneration.  
- **Sustainable Ecosystem Development**: Broad applicability in education, marketing, arts, and beyond.

---

## Conclusion and Future Tasks

This project proposes a new paradigm of personalized interactive storytelling.  
Future plans include model enhancement, UI/UX improvements, collaborations in education and cultural arts, and exploration of monetization strategies to achieve further advancement.